In [19]:
# import matplotlib.pyplot as plt
import numpy as np
import pybullet as p
import pybullet_data
import os
import time
import roboticstoolbox as rtb
import cv2
import spatialmath as sm
from spatialmath.base import delta2tr, tr2delta
# can use this to apply angular rotations to coordinate frames
from scipy.spatial.transform import Rotation as Rot

# camera (don't change these settings)
camera_width = 512  # image width
camera_height = 512  # image height
camera_fov = 120  # field of view of camera
camera_focal_depth = 0.5*camera_height/np.tan(0.5*np.pi/180*camera_fov)
# focal depth in pixel space
camera_aspect = camera_width/camera_height  # aspect ratio
camera_near = 0.02  # near clipping plane in meters, do not set non-zero
camera_far = 100  # far clipping plane in meters


# control objectives (if you wish, you can play with these values for fun)
object_location_desired = np.array([camera_width/2, camera_height/2])
# center the object to middle of image
K_p_x = 0.1  # Proportional control gain for translation
K_p_Omega = 0.02  # Proportional control gain for rotation


In [20]:
# Robot with Camera Class
class eye_in_hand_robot:
    def get_ee_position(self):
        '''
        Function to return the end-effector of the link. This is the very tip of the robot at the end of the jaws.
        '''
        endEffectorIndex = self.numActiveJoints
        endEffectorState = p.getLinkState(self.robot_id, endEffectorIndex)
        endEffectorPos = np.array(endEffectorState[0])
        endEffectorOrn = np.array(p.getMatrixFromQuaternion(
            endEffectorState[1])).reshape(3, 3)

        # add an offset to get past the forceps
        endEffectorPos += self.camera_offset*endEffectorOrn[:, 2]
        return endEffectorPos, endEffectorOrn

    def get_current_joint_angles(self):
        # Get the current joint angles
        joint_angles = np.zeros(self.numActiveJoints)
        for i in range(self.numActiveJoints):
            joint_state = p.getJointState(
                self.robot_id, self._active_joint_indices[i])
            joint_angles[i] = joint_state[0]
        return joint_angles

    def get_jacobian_at_current_position(self):
        # Returns the Robot Jacobian of the last active link
        mpos, mvel, mtorq = self.get_active_joint_states()
        zero_vec = [0.0]*len(mpos)
        linearJacobian, angularJacobian = p.calculateJacobian(self.robot_id,
                                                              self.numActiveJoints,
                                                              [0, 0, self.camera_offset],
                                                              mpos,
                                                              zero_vec,
                                                              zero_vec)
        # only return the active joint's jacobians
        Jacobian = np.vstack((linearJacobian, angularJacobian))
        return Jacobian[:, :self.numActiveJoints]

    def getJointStates(self):
        joint_states = p.getJointStates(
            self.robot_id, range(self._numLinkJoints))
        joint_positions = [state[0] for state in joint_states]
        joint_velocities = [state[1] for state in joint_states]
        joint_torques = [state[3] for state in joint_states]
        return joint_positions, joint_velocities, joint_torques

    def set_joint_position(self, desireJointPositions, kp=1.0, kv=0.3):
        zero_vec = [0.0] * self._numLinkJoints
        allJointPositionObjectives = [0.0]*self._numLinkJoints
        for i in range(desireJointPositions.shape[0]):
            idx = self._active_joint_indices[i]
            allJointPositionObjectives[idx] = desireJointPositions[i]

        p.setJointMotorControlArray(self.robot_id,
                                    range(self._numLinkJoints),
                                    p.POSITION_CONTROL,
                                    targetPositions=allJointPositionObjectives,
                                    targetVelocities=zero_vec,
                                    positionGains=[kp] * self._numLinkJoints,
                                    velocityGains=[kv] * self._numLinkJoints)

    def get_active_joint_states(self):
        joint_states = p.getJointStates(
            self.robot_id, range(self._numLinkJoints))
        joint_infos = [p.getJointInfo(self.robot_id, i)
                       for i in range(self._numLinkJoints)]
        joint_states = [j for j, i in zip(
            joint_states, joint_infos) if i[3] > -1]
        joint_positions = [state[0] for state in joint_states]
        joint_velocities = [state[1] for state in joint_states]
        joint_torques = [state[3] for state in joint_states]
        return joint_positions, joint_velocities, joint_torques

    def __init__(self, robot_id, initialJointPos):
        self.robot_id = robot_id
        self.eeFrameId = []
        self.camera_offset = 0.1  # offset camera in z direction to avoid grippers
        # Get the joint info
        self._numLinkJoints = p.getNumJoints(
            self.robot_id)  # includes passive joint
        jointInfo = [p.getJointInfo(self.robot_id, i)
                     for i in range(self._numLinkJoints)]

        # Get joint locations (some joints are passive)
        self._active_joint_indices = []
        for i in range(self._numLinkJoints):
            if jointInfo[i][2] == p.JOINT_REVOLUTE:
                self._active_joint_indices.append(jointInfo[i][0])
        # exact number of active joints
        self.numActiveJoints = len(self._active_joint_indices)

        # reset joints
        for i in range(self._numLinkJoints):
            p.resetJointState(self.robot_id, i, initialJointPos[i])


In [21]:
# Auxillary funcitons 

def operate_ee(robot,cmd):
    '''cmd = 0 to close, 1 to open'''
    # assert(cmd==0 or cmd==1)
    for I in range(50):
        p.setJointMotorControlArray(robot.robot_id, [9,10], p.POSITION_CONTROL, targetPositions=[cmd,cmd])
        p.stepSimulation()
        # global out
        # width, height, rgbImg, depthImg, segImg = p.getCameraImage(640, 480)
        # rgbImg = np.reshape(rgbImg, (height, width, 4))
        # out.write(cv2.cvtColor(rgbImg, cv2.COLOR_RGBA2BGR))

def move_ee(robot,T):
    
    ee_position = np.array(T)[0:3,3] 
    ee_orientation = sm.base.r2q(np.array(T)[0:3,0:3])
    pos_err = np.linalg.norm(robot.get_ee_position()[0]-ee_position)
    count=0
    while (pos_err> 0.01):
        j = np.array(p.calculateInverseKinematics(robot.robot_id,11,ee_position, ee_orientation))
        for i in range(len(j)):
            p.setJointMotorControl2(robot.robot_id, i, p.POSITION_CONTROL, j[i]) 
        p.stepSimulation()
        # global out
        # width, height, rgbImg, depthImg, segImg = p.getCameraImage(640, 480)
        # rgbImg = np.reshape(rgbImg, (height, width, 4))
        # out.write(cv2.cvtColor(rgbImg, cv2.COLOR_RGBA2BGR))
        time.sleep(1/240)
        pos_err = np.linalg.norm(robot.get_ee_position()[0]-ee_position)
        count=count+1
        if count>240:            
            break

def move_ee_timed(robot,T,t):
    ee_position = np.array(T)[0:3,3] 
    ee_orientation = sm.base.r2q(np.array(T)[0:3,0:3])
    
    for _ in range(t*240):

        j = np.array(p.calculateInverseKinematics(robot.robot_id,11,ee_position, ee_orientation))
        for i in range(len(j)):
            p.setJointMotorControl2(robot.robot_id, i, p.POSITION_CONTROL, j[i]) 
        p.stepSimulation()
        # global out
        # width, height, rgbImg, depthImg, segImg = p.getCameraImage(640, 480)
        # rgbImg = np.reshape(rgbImg, (height, width, 4))
        # out.write(cv2.cvtColor(rgbImg, cv2.COLOR_RGBA2BGR))
        time.sleep(1/240)
           

def move_traj_ee(robot,trajectory):
    for traj in trajectory:
        move_ee(robot,traj)

def manipulability_measure(jacobian):
    """
    Calculate the manipulability measure of a robot arm given its Jacobian matrix.
    :param jacobian: Jacobian matrix of the robot arm
    :return: manipulability measure
    """
    u, s, vh = np.linalg.svd(jacobian)
    return np.product(s)

def product_of_MM(robot1,robot2):
    return manipulability_measure(robot1.get_jacobian_at_current_position())*manipulability_measure(robot2.get_jacobian_at_current_position())

def to_optimise(z):
    global robot1,robot2
    T3 = sm.SE3.Rt(np.array(sm.SO3.RPY([90,0,90],unit= 'deg')),np.array([0.5,0.5,z]))
    T5 = sm.SE3.Rt(np.array(sm.SO3.RPY([0,-90,0],unit= 'deg')),np.array([0.5,0.5,z]))
    move_ee(robot1,T3)
    move_ee(robot2,T5)
    return -product_of_MM(robot1,robot2)

In [22]:
# Start the connection to the physics server

def pybenv():
    global robot1, robot2, box, out
    try:
        cv2.destroyAllWindows()
        p.disconnect()
    except:
        pass
    physicsClient = p.connect(p.GUI)  # (p.DIRECT)
    # time_step = 0.001
    p.resetSimulation()
    # p.setTimeStep(time_step)
    p.setGravity(0, 0, -9.8)

    # Set the path to the URDF files included with PyBullet
    p.setAdditionalSearchPath(pybullet_data.getDataPath())

    # load a plane URDF
    p.loadURDF('plane.urdf')
    
    p.resetDebugVisualizerCamera(
        cameraDistance=1, cameraYaw=30, cameraPitch=-52, cameraTargetPosition=[0, 0, .5])
    
    # Load robot 1
    pandaUid1 = p.loadURDF(os.path.join(
        pybullet_data.getDataPath(), "franka_panda\\panda.urdf"), useFixedBase=True)
    p.resetBasePositionAndOrientation(pandaUid1, [0, 0, 0], [0, 0, 0, 1])
    initialJointPosition = [0, -np.pi/4, np.pi/4, -
                            np.pi/4, np.pi/4, np.pi/4, np.pi/4, 0, 0, 0, 0, 0]
    robot1 = eye_in_hand_robot(pandaUid1, initialJointPosition)

    # Load robot 2
    pandaUid2 = p.loadURDF(os.path.join(
        pybullet_data.getDataPath(), "franka_panda\\panda.urdf"), useFixedBase=True)
    p.resetBasePositionAndOrientation(pandaUid2, [1, 0, 0], [0, 0, 0, 1])
    robot2 = eye_in_hand_robot(pandaUid2, initialJointPosition)

    # Load Box
    box = p.loadURDF("cube_small.urdf", [0,0.5, 0.02])
    for _ in range(240):
        p.stepSimulation()
        # width, height, rgbImg, depthImg, segImg = p.getCameraImage(640, 480)
        # rgbImg = np.reshape(rgbImg, (height, width, 4))
        # out.write(cv2.cvtColor(rgbImg, cv2.COLOR_RGBA2BGR))
        # time.sleep(1./240.)

    # Enable collision detection between robot and box
    p.setCollisionFilterPair(robot1.robot_id, box, -1, -1, 1)
    p.setCollisionFilterPair(robot2.robot_id, box, -1, -1, 1)



In [23]:
# # Set up video writer # uncomment necessory lines to record a video
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter('output.mp4', fourcc, 30.0, (640, 480))

In [24]:
pybenv()

In [25]:
# Optimizing the z coordinate for hand over
from scipy.optimize import minimize_scalar
operate_ee(robot1,1)
operate_ee(robot2,1)
res = minimize_scalar(to_optimise, bounds=(0.15, 0.75), method='bounded')
max_value = -1 * res.fun
max_z = res.x

print(f"Maximum value: {max_value}")
print(f"z value at maximum: {max_z}")

Maximum value: 0.00813641192960446
z value at maximum: 0.15003061403086104


In [26]:
# Moving the arms to a default position
T0 = sm.SE3.Rt(np.array(sm.SO3.RPY([90,0,0],unit= 'deg')),np.array([0,0.5,0.5]))
T7 = sm.SE3.Rt(np.array(sm.SO3.RPY([0,0,0],unit= 'deg')),np.array([1,0.5,0.5]))

move_ee(robot1,T0)
move_ee(robot2,T7)

In [27]:
# Moving robot1 towards the cube
T1 = sm.SE3.Rt(np.array(sm.SO3.RPY([90,0,0],unit= 'deg')),np.array([0,0.5,0.5]))
T2 = sm.SE3.Rt(np.array(sm.SO3.RPY([90,0,0],unit= 'deg')),np.array([0,0.5,0.025]))
trajectory =rtb.ctraj(T1,T2,20)
move_traj_ee(robot1,trajectory)

In [28]:
# Grasping the cube
operate_ee(robot1,0.017)

In [29]:
# Getting the robots to the hand over position
T3 = sm.SE3.Rt(np.array(sm.SO3.RPY([90,0,90],unit= 'deg')),np.array([0.5,0.5,max_z]))
trajectory =rtb.ctraj(T2,T3,20)
move_traj_ee(robot1,trajectory)

In [30]:
T4 = sm.SE3.Rt(np.array(sm.SO3.RPY([0,-90,0],unit= 'deg')),np.array([0.6,0.5,max_z]))
move_ee(robot2,T4)

In [31]:
operate_ee(robot2,1)
T5 = sm.SE3.Rt(np.array(sm.SO3.RPY([0,-90,0],unit= 'deg')),np.array([0.5,0.5,max_z]))
trajectory =rtb.ctraj(T4,T5,20)
move_traj_ee(robot2,trajectory)

In [32]:
# Hand over
operate_ee(robot2,0.017)
operate_ee(robot1,1)

In [33]:
# Placing the cube at the end position
T6 = sm.SE3.Rt(np.array(sm.SO3.RPY([0,0,0],unit= 'deg')),np.array([1,0.5,0.025]))
trajectory =rtb.ctraj(T5,T6,20)
move_traj_ee(robot2,trajectory)

In [34]:
operate_ee(robot2,1)

In [35]:
# Moving the arms at the default position
T7 = sm.SE3.Rt(np.array(sm.SO3.RPY([0,0,0],unit= 'deg')),np.array([1,0.5,0.5]))
move_ee(robot2,T7)
move_ee(robot1,T0)

In [36]:
# # Release video writer
# out.release()

In [39]:
try:
    cv2.destroyAllWindows()
    p.disconnect()
except:
    pass